In [41]:
import torch

print(torch.tensor([[1, 2],[3, 4]]))
# print(torch.tensor([[1, 2],[3, 4]], device="cuda:0"))
print(torch.tensor([[1, 2],[3, 4]], dtype=torch.float64))

tensor([[1, 2],
        [3, 4]])
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


In [42]:
# Numpy로 변환
temp = torch.tensor([[1, 2],[3, 4]])
print(temp.numpy())

[[1 2]
 [3 4]]


In [43]:
temp = torch.FloatTensor([1,2,3,4,5,7,8])
print(temp[0], temp[1], temp[-1])
print('------------------------')
print(temp[2:5], temp[2:-1])

tensor(1.) tensor(2.) tensor(8.)
------------------------
tensor([3., 4., 5.]) tensor([3., 4., 5., 7.])


In [44]:
# Change tensor dim
# -1은 유추
temp = torch.tensor([[1,2],[3,4]])
print('------------------------')
print(temp.view(4,1))
print('------------------------')
print(temp.view(-1))
print('------------------------')
print(temp.view(1,-1))
print('------------------------')
print(temp.view(-1,1))

------------------------
tensor([[1],
        [2],
        [3],
        [4]])
------------------------
tensor([1, 2, 3, 4])
------------------------
tensor([[1, 2, 3, 4]])
------------------------
tensor([[1],
        [2],
        [3],
        [4]])


In [45]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

In [57]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import requests

download_root = './data'

train_dataset = MNIST(download_root,transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root,transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root,transform=mnist_transform, train=False, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [47]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc = nn.Linear(7*7*64, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [48]:
model = MLP()
print(model)

MLP(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)


In [49]:
import torch.optim as optim

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [50]:
# 학습 설정
epochs = 5  # 반복 학습 횟수

for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 전환
    running_loss = 0.0

    for images, labels in train_loader:
        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    # 검증 단계
    # parameter 업데이트 X
    model.eval()
    correct = 0
    total = 0
    # back propagation X
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Epoch [1/5], Loss: 0.1833
Validation Accuracy: 97.55%
Epoch [2/5], Loss: 0.0550
Validation Accuracy: 98.80%
Epoch [3/5], Loss: 0.0404
Validation Accuracy: 98.93%
Epoch [4/5], Loss: 0.0319
Validation Accuracy: 98.95%
Epoch [5/5], Loss: 0.0255
Validation Accuracy: 99.09%
